In [29]:
import os
path = "C:/NIKEN/STIS/SEMESTER 5/Information Retrieval (P)/P2/berita"
for file in os.listdir(path):
    if os.path.isfile(os.path.join(path, file)):
        def read_text_file(file_path): 
            with open(file_path, 'r') as f:
                return f.read()

In [30]:
berita1 = read_text_file(f"{path}\\berita1.txt")
berita2 = read_text_file(f"{path}\\berita2.txt")
berita3 = read_text_file(f"{path}\\berita3.txt")
berita4 = read_text_file(f"{path}\\berita4.txt")
berita5 = read_text_file(f"{path}\\berita5.txt")

## Okapi BM 25

In [31]:
from rank_bm25 import BM25Okapi

def tokenisasi(text):
    tokens = text.split(" ")
    return tokens

def stemming(text):
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    
    #create stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    #stemming process
    output = stemmer.stem(text)
    return output

def stemming_sentence(text):
    output = ""
    for token in tokenisasi(text):
        output = output + stemming(token) + " "
    return output[:-1]

doc_dict_raw = {}
doc_dict_raw['doc1'] = berita1
doc_dict_raw['doc2'] = berita2
doc_dict_raw['doc3'] = berita3
doc_dict_raw['doc4'] = berita4
doc_dict_raw['doc5'] = berita5

doc_dict = {}
for doc_id, doc in doc_dict_raw.items():
    doc_dict[doc_id] = stemming_sentence(doc)

tokenized_corpus = [tokenisasi(doc_dict[doc_id]) for doc_id in doc_dict]

'''EXACT TOP-K'''
query = "vaksin corona jakarta"
tokenized_query = tokenisasi(query)
bm25 = BM25Okapi(tokenized_corpus)

doc_scores = bm25.get_scores(tokenized_query)

from collections import OrderedDict
def exact_top_k(doc_dict, rank_score, k):
    relevance_scores = {}
    i = 0
    for doc_id in doc_dict.keys():
        relevance_scores[doc_id] = rank_score[i]
        i = i + 1
        
    sorted_value = OrderedDict(sorted(relevance_scores.items(), key = lambda x: x[1], reverse = True))
    top_k = {j: sorted_value[j] for j in list(sorted_value)[:k]}
    return top_k

top_3 = exact_top_k(doc_dict, doc_scores, 3)
print(top_3)

{'doc3': 1.1034159549468965, 'doc2': 0.7542259391529292, 'doc5': 0.5204049944065936}


## Vector Space Model

In [32]:
def tokenisasi(text):
    tokens = text.split(" ")
    return tokens

def stemming(text):
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    # create stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    # stemming process
    output = stemmer.stem(text)
    return output

def stemming_sentence(text):
    output = ""
    for token in tokenisasi(text):
        output = output + stemming(token) + " "
    return output[:-1]

doc_dict_raw = {}
doc_dict_raw['doc1'] = berita1
doc_dict_raw['doc2'] = berita2
doc_dict_raw['doc3'] = berita3
doc_dict_raw['doc4'] = berita4
doc_dict_raw['doc5'] = berita5

doc_dict = {}
for doc_id,doc in doc_dict_raw.items():
    doc_dict[doc_id] = stemming_sentence(doc)

vocab = []
inverted_index = {}
for doc_id,doc in doc_dict.items():
    for token in tokenisasi(doc):
        if token not in vocab:
            vocab.append(token)
            inverted_index[token] = []
        if token in inverted_index:
            if doc_id not in inverted_index[token]:
                inverted_index[token].append(doc_id)

query = "vaksin corona jakarta"
def termFrequency(vocab, query):
    tf_query = {}
    for word in vocab:
        tf_query[word] = query.count(word)
    return tf_query
tf_query = termFrequency(vocab, query)

def wordDocFre(vocab, doc_dict):
    df = {}
    for word in vocab:
        frq = 0
        for doc in doc_dict.values():
            if word in tokenisasi(doc):
                frq = frq + 1
        df[word] = frq
    return df

'''TERM-FREQUENCY'''
def termFrequencyInDoc(vocab, doc_dict) :
    tf_docs = {}
    for doc_id in doc_dict.keys() :
        tf_docs[doc_id] = {}
    for word in vocab :
        for doc_id, doc in doc_dict.items() :
            tf_docs[doc_id][word] = doc.count(word)
    return tf_docs

import numpy as np
def inverseDocFre(vocab, doc_fre, length):
    idf = {}
    for word in vocab:
        #idf[word]  = np.log((length) / doc_fre[word])
        idf[word]  = 1 + np.log10((length + 1) / (doc_fre[word]+1))
    return idf

idf = inverseDocFre(vocab, wordDocFre(vocab, doc_dict), len(doc_dict))

# Term - Query Matrix
TQ = np.zeros((len(vocab), 1)) #hanya 1 query
for word in vocab:
    ind1 = vocab.index(word)
    TQ[ind1][0] = tf_query[word]*idf[word]

'''TF-IDF'''
def tfidf(vocab, tf, idf_scr, doc_dict) :
    tf_idf_scr = {}
    for doc_id in doc_dict.keys() :
        tf_idf_scr[doc_id] = {}
    for word in vocab :
        for doc_id, doc in doc_dict.items() :
            tf_idf_scr[doc_id][word] = tf[doc_id][word] * idf_scr[word]
    return tf_idf_scr

tf_idf = tfidf(vocab, termFrequencyInDoc(vocab, doc_dict), inverseDocFre(vocab, wordDocFre(vocab, doc_dict), len(doc_dict)), doc_dict)
    
'''TERM-DOCUMENT MATRIX'''
TD = np.zeros((len(vocab), len(doc_dict)))
for word in vocab :
    for doc_id, doc in tf_idf.items() :
        ind1 = vocab.index(word)
        ind2 = list(tf_idf.keys()).index(doc_id)
        TD[ind1][ind2] = tf_idf[doc_id][word]
        
'''COSINE SIMILARITY'''
import math
def cosine_sim(vec1, vec2) :
    vec1 = list(vec1)
    vec2 = list(vec2)
    dot_prod = 0
    for i, v in enumerate(vec1) :
        dot_prod += v * vec2[i]
    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))
    return dot_prod / (mag_1 * mag_2)

'''EXACT TOP-K'''
idf = inverseDocFre(vocab,wordDocFre(vocab,doc_dict),len(doc_dict))
query = "vaksin corona jakarta"
def termFrequency(vocab, query):
    tf_query = {}
    for word in vocab:
        tf_query[word] = query.count(word)
    return tf_query
tf_query = termFrequency(vocab, query)

from collections import OrderedDict
def exact_top_k1(doc_dict, TD, q, k):
    relevance_scores = {}
    i = 0
    for doc_id in doc_dict.keys():
        relevance_scores[doc_id] = cosine_sim(q, TD[:, i])
        i = i + 1
    sorted_value = OrderedDict(sorted(relevance_scores.items(), key=lambda x: x[1], reverse = True))
    top_k = {j: sorted_value[j] for j in list(sorted_value)[:k]}
    return top_k

In [33]:
top_3_2 = exact_top_k1(doc_dict, TD, TQ[:, 0], 3)
print("VECTOR SPACE MODEL TOP-3")
print(top_3_2)

VECTOR SPACE MODEL TOP-3
{'doc2': 0.16640342749946516, 'doc3': 0.13671100189084426, 'doc5': 0.06326722354821664}


## Likelihood Model

In [34]:
likelihood_scores = {}
vocab = set()
for doc_id in doc_dict.keys():
    likelihood_scores[doc_id] = 1
    tokens = tokenisasi(doc_dict[doc_id])
    vocab.update(tokens)
    for q in tokenized_query:
        likelihood_scores[doc_id]=likelihood_scores[doc_id]*tokens.count(q)/len(tokens)

print(likelihood_scores)


{'doc1': 0.0, 'doc2': 0.0, 'doc3': 3.756574004507889e-06, 'doc4': 0.0, 'doc5': 0.0}


In [35]:
from collections import OrderedDict
sorted_value = OrderedDict(sorted(likelihood_scores.items(), key=lambda x: x[1], reverse = True))
top_3 = {j: sorted_value[j] for j in list(sorted_value)[:3]}
print(top_3)


{'doc3': 3.756574004507889e-06, 'doc1': 0.0, 'doc2': 0.0}


#### Membuat Vocab

In [36]:
tokenized_corpus = [j for sub in [tokenisasi(doc_dict[doc_id]) for 
doc_id in doc_dict] for j in sub]
vocab = set(tokenized_corpus)
print(vocab)

{'fauci', 'mobilitas', 'detik', 'dokter', 'hijau', 'wilayah', 'oleh', 'tahap', 'harus', 'com', 'satgas', 'jawab', 'panas', 'kamu', 'level', 'yang', 'longgar', 'kait', 'nasihat', '165', 'memang', 'djoerban', 'ketua', 'area', 'asal', 'cs', 'sehat', 'rutin', '90', 'laut', 'belum', 'lantas', 'perlu', 'ppkm', 'untuk', 'd-5816582', 'tahu', 'dr', 'pasca', 'zona', 'batas', 'dadak', 'suntik', 'jika', 'apa', '24', 'prof', 'sebut', 'timur', 'corona', 'wilayah-kamu-sudah-bebas-covid-19-cek-34-kabkota-zona-hijau-terbaru', 'senin', 'musim', '1', 'vaksinasi', 'vaksin', 'langsung', 'ada', 'gantung', 'atau', 'ri', 'padahal', 'total', '11', 'picu', 'utara', 'puncak', 'jenis', 'usai', 'covid-19', '2021', 'siti', 'tidak', 'tengah', 'alami', 'virus', 'kota', 'kemudian', 'cukup', 'kembali', 'cegah', 'kemenkes', 'pakar', 'balitbangkes', 'efektivitas', 'juga', '13', 'varian', 'tular', 'mulai', 'pfizer', 'amerika', 'kab', 'november', 'medis', 'sekali', 'tingkat', 'sudah', 'bakal', 'seperti', 'ikut', 'alert', '

### Likelihood Model dengan Laplace Smoothing

In [37]:
alpha = 1
likelihood_scores_laplace = {}
for doc_id in doc_dict.keys():
    likelihood_scores_laplace[doc_id] = 1
    tokens = tokenisasi(doc_dict[doc_id])
    for q in tokenized_query:
        likelihood_scores_laplace[doc_id]=likelihood_scores_laplace[doc_id]*(tokens.count(q)+alpha)/(len(tokens)+len(vocab)*alpha)

print(likelihood_scores_laplace)

{'doc1': 5.990490455830797e-08, 'doc2': 2.4644382333070736e-07, 'doc3': 5.999746810684589e-07, 'doc4': 1.5402738958169208e-07, 'doc5': 1.1870046611002282e-07}


In [38]:
sorted_value = OrderedDict(sorted(likelihood_scores_laplace.items(), key=lambda x: x[1], reverse = True))
top_3 = {j: sorted_value[j] for j in list(sorted_value)[:3]}
print(top_3)

{'doc3': 5.999746810684589e-07, 'doc2': 2.4644382333070736e-07, 'doc4': 1.5402738958169208e-07}


### Likelihood Model dengan Jelinek-Mercer Smoothing

In [39]:
lamda = 0.5
likelihood_scores_mercer = {}
for doc_id in doc_dict.keys():
    likelihood_scores_mercer[doc_id] = 1
    tokens = tokenisasi(doc_dict[doc_id])
    for q in tokenized_query:
        likelihood_scores_mercer[doc_id]=likelihood_scores_mercer[doc_id]*((lamda*tokens.count(q)/len(tokens))+((1-lamda)*tokenized_corpus.count(q)/len(tokenized_corpus)))

print(likelihood_scores_mercer)


{'doc1': 2.6190692275527747e-07, 'doc2': 7.97496704914498e-07, 'doc3': 2.758125277898707e-06, 'doc4': 5.844033998799556e-07, 'doc5': 9.270498351757131e-07}


In [40]:
sorted_value = OrderedDict(sorted(likelihood_scores_mercer.items(), key=lambda x: x[1], reverse = True))
top_3 = {j: sorted_value[j] for j in list(sorted_value)[:3]}
print(top_3)

{'doc3': 2.758125277898707e-06, 'doc5': 9.270498351757131e-07, 'doc2': 7.97496704914498e-07}


### Likelihood Model dengan Dirichlet Smoothing

In [41]:
miu = 2
likelihood_scores_dirichlet = {}
for doc_id in doc_dict.keys():
    likelihood_scores_dirichlet[doc_id] = 1
    tokens = tokenisasi(doc_dict[doc_id])
    for q in tokenized_query:
        likelihood_scores_dirichlet[doc_id]=likelihood_scores_dirichlet[doc_id]*(tokens.count(q)+miu*tokenized_corpus.count(q)/len(tokenized_corpus))/(len(tokens)+miu)

print(likelihood_scores_dirichlet)


{'doc1': 3.931902225004399e-10, 'doc2': 3.8300390580787434e-08, 'doc3': 3.73857765656231e-06, 'doc4': 1.6938850281982751e-09, 'doc5': 4.4630340210217655e-08}


In [42]:
sorted_value = OrderedDict(sorted(likelihood_scores_dirichlet.items(), key=lambda x: x[1], reverse = True))
top_3 = {j: sorted_value[j] for j in list(sorted_value)[:3]}
print(top_3)

{'doc3': 3.73857765656231e-06, 'doc5': 4.4630340210217655e-08, 'doc2': 3.8300390580787434e-08}
